<a href="https://colab.research.google.com/github/Zain-Pirzada/Diabetic-Reinopathy-Kaggle-Competition/blob/main/Transfer_Learning_Cabnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from keras.applications import InceptionResNetV2
from keras.layers.core import Lambda
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
#from keras.utils import multi_gpu_model
from matplotlib import pyplot as plt
from keras.models import load_model
import keras.backend as K
import os


In [13]:

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

In [14]:
def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalMaxPool2D()(F2)
    
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=Multiply()([inputs,M])
    return semantic


In [15]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points    
   

In [16]:
def plotmodel(history,name):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1) 
    
    plt.figure(1)                  
    plt.plot(epochs,smooth_curve(acc))
    plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.savefig('acc_'+name+'.png')
    
    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper right')
    plt.savefig('loss_'+name+'.png')

In [17]:
def label_smooth(y_true, y_pred):
    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred) 
 

In [18]:
def get_base_model(model_name,image_size):
    if model_name =='vgg16':
        base_model=VGG16              (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='resnet50':
        base_model=ResNet50           (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='xception':
        base_model=Xception           (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='densenet121':
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='mobilenet0.75':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=0.75,input_shape=(image_size,image_size,3))
    if model_name =='mobilenet1.0':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='mobilenetv2':
        base_model=MobileNetV2      (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='inceptionv3':   
        base_model=InceptionV3       (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='inceptionv2':
        base_model=InceptionResNetV2 (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    return base_model

In [19]:
def train_model(model,dataset,image_size,batch_size,save_name,lr1,lr2,Epochs1,Epochs2):
    
    #dataParam={'messidor': [957,243,2,'./data/messidor/train','./data/messidor/test'],
              #  'kaggle': [30000,5126,5,'./data/kaggle/train','./data/kaggle/valid'],
              #  'DDR':   [9851,2503,5,'./data/DDR/train','./data/DDR/valid']} #6119
    dataParam = {'DR': [3200,800,5,'//content/drive/MyDrive/CABnet-master/data/val','//content/drive/MyDrive/CABnet-master/data/test']}

    train_num,valid_num,classes,train_dir,test_dir = dataParam[dataset]
    
    train=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=90)          
    valid = ImageDataGenerator()
    train_data=train.flow_from_directory(train_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = True,
                                         batch_size=batch_size)
    valid_data=valid.flow_from_directory(test_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = False,
                                         batch_size=batch_size)

    lr_decay=ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1)
    save_model=ModelCheckpoint('new/'+save_name+'{epoch:02d}.h5', monitor='val_loss',period=10)
    
    for layer in base_model.layers:
        layer.trainable = False   
        
    model.compile(optimizer=Adam(lr=lr1,decay=0.00001),loss=loss_fun,metrics=['acc'])
    model.fit_generator(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs1, 
                        workers=2,
                        callbacks=[lr_decay,save_model])   
    
    for layer in base_model.layers:
        layer.trainable = True
        
    model.compile(optimizer=Adam(lr=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])
    history=model.fit_generator(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs2,
                        workers=2,
                        callbacks=[lr_decay,save_model])
    return history


In [20]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"    
loss_fun= 'categorical_crossentropy'  
gpu_num=1
k=5
lr1=0.005
lr2=0.0001
batch_size= 32
image_size=224
classes=5

base_model=get_base_model('densenet121',image_size)  
base_in=base_model.input
base_out=base_model.output

x=Global_attention_block(base_out)
base_out=Category_attention_block(x,classes,k)

x=GlobalAveragePooling2D()(base_out)
out=Dense(classes,activation='softmax')(x)

# if gpu_num>1:
#     model=Model(base_model.input,out)
#     #model.summary()
#     parallel_model = multi_gpu_model(model, gpus=gpu_num)
#     parallel_model.summary()
# else:
parallel_model=Model(base_model.input,out)
parallel_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [ ]:
history=train_model(parallel_model,'DR',image_size,batch_size,'densenet121',lr1,lr2,1,70)
plotmodel(history,'densenet121')


Found 7023 images belonging to 5 classes.


Found 7030 images belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
<ipython-input-19-c84d6e4381b2>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,


100/100 [==============================] - 1191s 12s/step - loss: 0.9946 - acc: 0.7220 - val_loss: 0.5113 - val_acc: 1.0000 - lr: 0.0050


<ipython-input-19-c84d6e4381b2>:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_data,


Epoch 1/70
100/100 [==============================] - 2804s 28s/step - loss: 0.8581 - acc: 0.7284 - val_loss: 0.5954 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 2/70
100/100 [==============================] - 2796s 28s/step - loss: 0.8006 - acc: 0.7391 - val_loss: 0.4698 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 3/70
100/100 [==============================] - 2764s 28s/step - loss: 0.7727 - acc: 0.7394 - val_loss: 0.3202 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 4/70
100/100 [==============================] - 2752s 27s/step - loss: 0.7400 - acc: 0.7428 - val_loss: 0.2578 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 5/70
100/100 [==============================] - 2750s 27s/step - loss: 0.7368 - acc: 0.7421 - val_loss: 0.3771 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 6/70
100/100 [==============================] - 2739s 27s/step - loss: 0.7254 - acc: 0.7424 - val_loss: 0.1114 - val_acc: 1.0000 - lr: 1.0000e-04
Epoch 7/70
100/100 [==============================] - 2779s 28s/step - loss: 0.7487 - ac